# Chapter 03: Structured Outputs

Structured outputs are useful when you need your data to fit a specific format, ensuring it's easy to process, analyze, or store. Think of it like this: instead of getting a blob of text, you get neat, well-organized data that can be directly used in your applications. Here are some common output types that LLMs can generate:
- **String**: Simple text, like "Hello" or "World". 
- **Integer**: Whole numbers, such as `123` or `-456`. 
- **Float**: Numbers with decimals, like `3.14` or `-0.001`. 
- **Boolean**: True/False values, useful for binary decisions. 
- **Binary**: Data in binary format, something like `10101010`. 
- **Date**: Calendar dates in formats like `YYYY-MM-DD`. 
- **Timestamp**: Specific points in time like `2023-10-05 14:48:00`.
- **Array**: Lists of elements, which can be strings, integers, floats, structs, or even other arrays.
- **Struct**: Collections of key-value pairs. 
- **Object**: Objects with specified attributes. 

By generating structured outputs, LLMs can: 
- **Integrate with systems**: Easily slot into existing databases, APIs, or other software. 
- **Enable automation**: Power automated reporting, decision-making engines, and other workflows. 
- **Enhance accuracy**: Ensure data consistency and integrity.

In [1]:
import re
import dirtyjson as json
from typing import Any
from pydantic import BaseModel, EmailStr, Field, ValidationError
from datetime import datetime
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

To get a sense of structured outputs, we will walk through a few examples. When the final answer from the LLM is successfully parsed, it is crucial to validate the output to ensure it matches the expected output type. If the validation is successful, we can return the response to the user. If not, we ask the LLM to correct its output if we use Chain-of-Thought prompting.

**String**

The standard output type for LLM applications is typically a string. However, in certain cases, you might only want the LLM to output a single word or a brief phrase. Additionally, it's important to instruct the LLM on how to handle edge cases - situations where the input does not match the expected input described to the model. For string outputs, this can involve responding with an empty string when the expected input format is not met.

In [4]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a string"""

prompt = "What is the capital city of France?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

Paris


**Integer**

For integer output types, the LLM should be instructed to provide a numerical response within a specified range, extract numbers from the input text, or output a value based on the results from a tool it uses. In the event of unexpected input, it should return a default value, such as 0 or -1.

In [5]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be an integer"""

prompt = "How many continents are there on Earth?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

7


In [6]:
try:
    output = int(output)
    print(output)
except ValueError as error:
    print(error)

7


**Float**

For float output types, the LLM should generate a numerical response with decimal precision, either within a specified range or based on extracted data from input text or tool outputs. In case of unexpected input, it should return a default value, like 0.0 or -1.0.

In [7]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be an float"""

prompt = "What is the value of Pi up to two decimal places?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

3.14


In [8]:
try:
    output = float(output)
    print(output)
except ValueError as error:
    print(error)

3.14


**Boolean**

For boolean output types, the LLM should return "true" or "false" based on the evaluation of input data or tool outputs. For example, when asked whether the contractor has violated terms, the response should be "true" if there is evidence of a violation and "false" otherwise.

In [9]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a boolean (true, false)"""

prompt = "Is the number 5 greater than 3?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

True


In [10]:
if output.lower() == "true":
    print(True)
elif output.lower() == "false":
    print(False)
else:
    print("Could not parse output")

True


**Binary**

A binary string output in LLM applications can offer a compact, universally understandable format for encoding data, which is particularly useful for tasks requiring data serialization, cryptographic applications, or low-level hardware communication.

In [11]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a binary string"""

prompt = "What is the binary representation of the number 15?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

1111


In [12]:
if bool(re.fullmatch(r"[01]+", output)):
    print(output)
else:
    print("Could not parse output")

1111


**Date**

A date output type is essential for accurately managing and processing time-related information, facilitating tasks such as scheduling, temporal queries, and event tracking. It ensures consistent and standardized handling of dates, which is crucial for applications in domains like finance, healthcare, and project management.

In [13]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract the date from the user's input text.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a date with the format: %Y-%m-%d"""

prompt = "We are excited to announce that our annual company retreat will be held on April 15, 2024. This event will be a great opportunity for team building and strategic planning."

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

2024-04-15


In [14]:
try:
    output = datetime.strptime(output, "%Y-%m-%d")
    print(output)
except ValueError as error:
    print(error)

2024-04-15 00:00:00


**Timestamp**

A timestamp output type is valuable for precisely recording and managing events with specific dates and times, which is crucial for activities like logging transactions, tracking changes, and conducting time-sensitive data analysis. This precision supports robust auditing, accurate scheduling, and reliable temporal sequencing across various use cases.

In [15]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract the date from the user's input text.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a date with the format: %Y-%m-%d %H:%M:%S"""

prompt = "We are excited to announce that our annual company retreat will be held on April 15, 2024. This event will be a great opportunity for team building and strategic planning."

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

2024-04-15 00:00:00


In [16]:
try:
    output = datetime.strptime(output, "%Y-%m-%d %H:%M:%S")
    print(output)
except ValueError as error:
    print(error)

2024-04-15 00:00:00


**Array**

In LLM applications, an array output type is crucial for efficiently managing sequences of tokens, data structures, or lists of generated items, enabling the model to handle complex outputs such as ordered collections, tables, and multi-step instructions more effectively.

In [17]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract all numbers from the user's input text.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be an array of integers"""

prompt = """Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.

By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.

We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

[6, 15, 7, 10, 4401, 2, 12, 5, 6, 12, 5]


In [18]:
try:
    output = json.loads(output)
    output = list(output)
    if all(isinstance(entry, int) for entry in output):
        print(output)
    else:
        raise ValueError("Could not parse output")
except Exception as error:
    print(error)

[6, 15, 7, 10, 4401, 2, 12, 5, 6, 12, 5]


**Struct**

A struct output type groups different kinds of data as key-value pairs under one name, making it easier to organize and access complex information.

In [19]:
def get_schema_from_args(args: dict[str, Any]) -> dict[str, Any]:
    template = '    "{field}": {field_type}, # {description}'
    fields = []
    for field, details in args.items():
        field_type = details.get("type")
        items_type = details.get("items", {}).get("type")
        format_type = details.get("items", {}).get("format") or details.get("format")
        description = details.get("description")
        if field_type in "string":
            if format_type == "date":
                fields.append(template.format(field=field, field_type="<date>", description=description))
            elif format_type == "date-time":
                fields.append(template.format(field=field, field_type="<timestamp>", description=description))
            elif format_type == "email":
                fields.append(template.format(field=field, field_type="<email>", description=description))
            else:
                fields.append(template.format(field=field, field_type="<string>", description=description))
        elif field_type == "integer":
            fields.append(template.format(field=field, field_type="<integer>", description=description))
        elif field_type == "number":
            fields.append(template.format(field=field, field_type="<float>", description=description))
        elif field_type == "boolean":
            fields.append(template.format(field=field, field_type="<boolean>", description=description))
        elif field_type == "array":
            if items_type == "string":
                if format_type == "date":
                    fields.append(template.format(field=field, field_type="<array of dates>", description=description))
                elif format_type == "date-time":
                    fields.append(template.format(field=field, field_type="<array of timestamps>", description=description))
                elif format_type == "email":
                    fields.append(template.format(field=field, field_type="<array of emails>", description=description))
                else:
                    fields.append(template.format(field=field, field_type="<array of strings>", description=description))
            elif items_type == "integer":
                fields.append(template.format(field=field, field_type="<array of integers>", description=description))
            elif items_type == "number":
                fields.append(template.format(field=field, field_type="<array of floats>", description=description))
    schema = "\n".join(fields)
    return "\n".join(["{", schema, "}"])

In [20]:
class Email(BaseModel):
    to: list[EmailStr] = Field(description="List of people to send it to")
    subject: str = Field(description="Subject of the email")
    body: str = Field(description="Body of the email")

args = Email.model_json_schema()["properties"]
output_schema = get_schema_from_args(args)
print(output_schema)

{
    "to": <array of emails>, # List of people to send it to
    "subject": <string>, # Subject of the email
    "body": <string>, # Body of the email
}


In [21]:
system_prompt = f"""You are an AI assistant designed to help users with a variety of tasks.

Write an email.

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```

Your <response to the prompt> should be the final answer to the user's query and must be a JSON format with the keyword arguments:
{output_schema}"""

prompt = """Send the email to:
1. johndoe@example.com
2. janedoe@example.com
3. alice.smith@company.org

Here is what we did: Weekend Hiking Trip Recap

Here is some context:
Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.
By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.
We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt)
])

print(output)

{
    "to": ["johndoe@example.com", "janedoe@example.com", "alice.smith@company.org"],
    "subject": "Weekend Hiking Trip Recap",
    "body": "Hello all,\n\nI hope this email finds you well. I am writing to recap our amazing hiking trip last weekend.\n\nWe started our 15-kilometer hike at 7 AM. By noon, we had covered 10 kilometers. We reached the summit of Mount Elbert, standing tall at 4,401 meters, by 2 PM. The weather was quite pleasant with a temperature of 12°C.\n\nWe set up our camp 5 kilometers away from the summit by 6 PM. We were joined by 12 other fellow hikers at the campsite. After spending a night under the stars, we packed up and returned home by 5 PM the next day.\n\nIt was an unforgettable experience and I am looking forward to our next adventure.\n\nBest,\n[Your Name]"
}


In [22]:
try:
    output = json.loads(output)
    output = Email.model_validate(output)
    print(output.model_dump())
except ValidationError as error:
    print(error)

{'to': ['johndoe@example.com', 'janedoe@example.com', 'alice.smith@company.org'], 'subject': 'Weekend Hiking Trip Recap', 'body': 'Hello all,\n\nI hope this email finds you well. I am writing to recap our amazing hiking trip last weekend.\n\nWe started our 15-kilometer hike at 7 AM. By noon, we had covered 10 kilometers. We reached the summit of Mount Elbert, standing tall at 4,401 meters, by 2 PM. The weather was quite pleasant with a temperature of 12°C.\n\nWe set up our camp 5 kilometers away from the summit by 6 PM. We were joined by 12 other fellow hikers at the campsite. After spending a night under the stars, we packed up and returned home by 5 PM the next day.\n\nIt was an unforgettable experience and I am looking forward to our next adventure.\n\nBest,\n[Your Name]'}


**Object**

An object output type enables structured data representation, making it easier to parse, manipulate, and integrate with other systems or workflows, thereby improving efficiency and reducing the potential for errors in downstream applications.

In [23]:
try:
    output = Email.model_validate(output)
    print(f"To: {output.to}")
    print(f"Subject: {output.subject}")
    print(f"Body: {output.body}")
except ValidationError as error:
    print(error)

To: ['johndoe@example.com', 'janedoe@example.com', 'alice.smith@company.org']
Subject: Weekend Hiking Trip Recap
Body: Hello all,

I hope this email finds you well. I am writing to recap our amazing hiking trip last weekend.

We started our 15-kilometer hike at 7 AM. By noon, we had covered 10 kilometers. We reached the summit of Mount Elbert, standing tall at 4,401 meters, by 2 PM. The weather was quite pleasant with a temperature of 12°C.

We set up our camp 5 kilometers away from the summit by 6 PM. We were joined by 12 other fellow hikers at the campsite. After spending a night under the stars, we packed up and returned home by 5 PM the next day.

It was an unforgettable experience and I am looking forward to our next adventure.

Best,
[Your Name]
